# [Module 6] Personalize 솔루션 버전 평가 지표 확인

이 노트북은 아래와 같은 작업을 합니다.
- 솔루션 버전으로부터 성능 지표 구하기
- 성능 지표 분석

## 0. 환경 설정

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

## 솔루션 버전 평가를 위한 학습 데이터 분리 방법

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다.

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.

![personalize metrics](img/personalize_metrics.png)

### 솔루션 버전의 성능 지표 구하기

In [4]:
metrics=[]

def build_metric_matrix(solution,response):
    metrics.append([solution,
                response['metrics']['coverage'],
                response['metrics']['mean_reciprocal_rank_at_25'],
                response['metrics']['normalized_discounted_cumulative_gain_at_5'],
                response['metrics']['normalized_discounted_cumulative_gain_at_10'],
                response['metrics']['normalized_discounted_cumulative_gain_at_25'],
                response['metrics']['precision_at_5'],
                response['metrics']['precision_at_10'],
                response['metrics']['precision_at_25']])

### Metrics : USER-PERSONALIZATION

In [5]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = base_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('user_personalization',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:376278017302:solution/RetailDemo-user-personalization/02b4a057",
  "metrics": {
    "coverage": 0.8852,
    "mean_reciprocal_rank_at_25": 0.4206,
    "normalized_discounted_cumulative_gain_at_10": 0.4368,
    "normalized_discounted_cumulative_gain_at_25": 0.4998,
    "normalized_discounted_cumulative_gain_at_5": 0.3725,
    "precision_at_10": 0.1317,
    "precision_at_25": 0.072,
    "precision_at_5": 0.1913
  },
  "ResponseMetadata": {
    "RequestId": "e77d745b-ee1c-4156-aef2-88d2231147cc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 20 Jun 2023 11:15:11 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "414",
      "connection": "keep-alive",
      "x-amzn-requestid": "e77d745b-ee1c-4156-aef2-88d2231147cc"
    },
    "RetryAttempts": 0
  }
}


## Summary Metrics

#### 1. 솔루션 평가 지표 정의
[솔루션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔루션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)

#### 2. 솔류션 버전의 성능 지표 분석

레서피의 종류는 크게 세가지 입니다. 각각에 대해 확인을 해보겠습니다.
#### 1. USER_PERSONALIZATION Recipes
- 5가지 (popularity,user_personalization,hrnn,hrnn_meta,hrnn_coldstart) 있습니다.
- popularity 는 베이스라인의 레서피로서 샤용을 주로 합니다. 모든 지표에서 가장 낮은 수치를 보입니다.
- hrnn, hrnn-meta, user_personalization 이 전반적으로 가장 높은 성능을 보여 줍니다. user_personalization은 default로 exploration_weight=0.3 입니다. coldstart item 이 거의 없는 상태에서도 높은 성능을 보여 주고 있습니다.학습 데이터의 인터렉션 데이터에 콜드 스타트 아이템이 전형 없기에 성능은 감소합니다. 이 경우에는 exloration_weight = 0을 하게 되면 콜드 스타트의 아이템을 배제하고 추천 합니다.<br>
이 링크 [exploration_weight](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)에서 "Item Exploration Configuration Hyperparameters" 이 부분을 참고 하세요.<br>


In [6]:
recipe_metrics=pd.DataFrame(metrics,columns=['recipe','coverage','mrr@25','ndcg@5','ndcg@10','ndcg@25','p@5','p@10','p@25'])

recipe_metrics

,recipe,coverage,mrr@25,ndcg@5,ndcg@10,ndcg@25,p@5,p@10,p@25
0,user_personalization,0.8852,0.4206,0.3725,0.4368,0.4998,0.1913,0.1317,0.072
